# Pandemic Simulation with Object-Oriented Programming

## Table of Content

1. [Representing Social Connections](#1)
2. [Simulate Disease Spread](#2)
3. [Plot graph to visualize the simulation](#3)

### 1. Representing Social Connections <a class="anchor" id="1"></a>

### 2. Simulate Disease Spread <a class="anchor" id="2"></a>

### 3. Plot graph to visualize the simulation <a class="anchor" id="3"></a>